In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import uproot

def read_root_to_pd(file_path):
    """读取ROOT文件中的ntuple数据到pandas DataFrame"""
    file = uproot.open(file_path)
    ntuple = file["ntuple"]
    return ntuple.arrays(library="pd")

# 读取两个文件
exp_df = read_root_to_pd("/home/zhonghua/data/Dataset_Filted/AExp_params_6m5fpd.root")
mc_df = read_root_to_pd("/home/zhonghua/data/Dataset_Filted/Proton_train_60.root")  # train data to give weight
mc_test_df = read_root_to_pd("/home/zhonghua/data/Dataset_Filted/Proton_closure_40.root") # closure test data


# 计算R_ue
ratio_exp = (exp_df["NuM1"] + 1e-4) / (exp_df["NpE3"] + 1)
ratio_mc = (mc_df["NuM1"] + 1e-4) / (mc_df["NpE3"] + 1)
ratio_mc_test = (mc_test_df["NuM1"] + 1e-4) / (mc_test_df["NpE3"] + 1)
exp_df["R_ue"] = np.log10(np.clip(ratio_exp, 1e-8, None))  # 避免log(0)
mc_df["R_ue"] = np.log10(np.clip(ratio_mc, 1e-8, None))
mc_test_df["R_ue"] = np.log10(np.clip(ratio_mc_test, 1e-8, None))

print(f"数据集1: {exp_df.shape}")
print(f"数据集2: {mc_df.shape}")
print(f"数据集3: {mc_test_df.shape}")
print(exp_df.columns)
print(mc_df.columns)

sample_num= 1e6
exp_df_sample = exp_df.sample(n=int(sample_num))
mc_df_sample = mc_df.sample(n=int(sample_num))
mc_test_df_sample = mc_test_df.sample(n=int(sample_num))
# 释放原始df
del exp_df, mc_df, mc_test_df



数据集1: (79075106, 22)
数据集2: (2184367, 23)
数据集3: (1456245, 23)
Index(['mjd', 'r_Theta', 'r_Phi', 'r_Corex', 'r_Corey', 'NhitE', 'NfiltE',
       'NtrigE', 'NhitM', 'NfiltM', 'NpE1', 'NpE2', 'NpE3', 'NuM1', 'NuM2',
       'NuM3', 'size', 'age', 'dr', 'NuW2', 'recE', 'R_ue'],
      dtype='object')
Index(['mjd', 'r_Theta', 'r_Phi', 'r_Corex', 'r_Corey', 'NhitE', 'NfiltE',
       'NtrigE', 'NhitM', 'NfiltM', 'NpE1', 'NpE2', 'NpE3', 'NuM1', 'NuM2',
       'NuM3', 'size', 'age', 'dr', 'NuW2', 'recE', 'weight', 'R_ue'],
      dtype='object')
数据集1: (1000000, 22)
数据集2: (1000000, 23)
数据集3: (1000000, 23)


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tqdm import tqdm
from scipy.stats import chi2

# ---------- 配置区（按需修改） ----------
vars_features = ["r_Theta", "age", "dr"]   # 用于 reweight 的特征（control region）
label_exp = 1
label_mc = 0

# Classifier 超参（可调整）
clf_args = dict(n_estimators=300, learning_rate=0.1, max_depth=4, random_state=42)

# Calibration method: 'isotonic' or 'sigmoid'
calibration_method = 'isotonic'

# 权重剪切与归一化参数
clip_min = 1e-3
clip_max = 50.0   # 上限，防止单事件主导
normalize_mc_total = True   # 将 reweighted MC 保持与原始 MC total weight 相同（常用）

# bootstrap 设置
Nboot = 300
random_seed = 12345
n_jobs = 4   # 若使用并行训练的模型可设置

# closure 测试相关
closure_var = "recE"
rue_edges = np.linspace(np.percentile(exp_df[closure_var], 1),
                        np.percentile(exp_df[closure_var], 99), 50)
cut_low, cut_high = 1.79, 2.13
# ----------------------------------------

rng_global = np.random.default_rng(random_seed)

# ---------- 辅助函数 ----------
def train_reweighter(X_mc_train, X_exp_train, w_mc_train=None):
    """
    Train classifier to separate data vs mc.
    Inputs:
      X_mc_train: (N_mc_train, n_feat) numpy array
      X_exp_train: (N_exp_train, n_feat)
      w_mc_train: original mc weights for training (or None -> ones)
    Returns:
      calibrator: calibrated classifier object with predict_proba
      clf_base: raw classifier (optional)
    """
    X = np.vstack([X_exp_train, X_mc_train])
    y = np.concatenate([np.ones(len(X_exp_train)), np.zeros(len(X_mc_train))])
    # sample weights: data weight = 1, mc weight = w_mc_train (if provided)
    if w_mc_train is None:
        sw = np.concatenate([np.ones(len(X_exp_train)), np.ones(len(X_mc_train))])
    else:
        sw = np.concatenate([np.ones(len(X_exp_train)), w_mc_train])

    # train-test split for calibration set
    X_tr, X_val, y_tr, y_val, sw_tr, sw_val = train_test_split(
        X, y, sw, test_size=0.25, random_state=random_seed, stratify=y)

    clf = GradientBoostingClassifier(**clf_args)
    clf.fit(X_tr, y_tr, sample_weight=sw_tr)

    # calibrate probabilities on validation set
    calibrator = CalibratedClassifierCV(base_estimator=clf, method=calibration_method, cv='prefit')
    calibrator.fit(X_val, y_val, sample_weight=sw_val)
    return calibrator, clf

def get_reweight_from_model(model, X_mc_eval, w_mc_orig=None, clip=(clip_min, clip_max), normalize=normalize_mc_total):
    """
    Compute event-wise reweight for MC eval set.
    reweight = p/(1-p) where p = model.predict_proba(X)[:,1]
    """
    p = model.predict_proba(X_mc_eval)[:,1]
    eps = 1e-6
    p = np.clip(p, eps, 1-eps)
    w_rew = p / (1.0 - p)

    # optional: multiply by original mc weight if provided (we keep them separate)
    if w_mc_orig is None:
        w = w_rew
    else:
        w = w_rew * w_mc_orig

    # clip extreme values
    w = np.clip(w, clip[0], clip[1])

    if normalize and (w_mc_orig is not None):
        # keep total MC weight same as original total (shape correction only)
        total_orig = np.sum(w_mc_orig)
        total_new = np.sum(w)
        if total_new > 0:
            w = w * (total_orig / total_new)
    return w, w_rew  # return both final applied weights and raw reweight

def weighted_hist_and_uncertainty(vals, weights, bins):
    """
    Fast weighted histogram + per-bin sum of squared weights (for variance)
    Returns:
      hist (sum weights per bin), w2_per_bin (sum weights^2)
    """
    bin_idx = np.digitize(vals, bins) - 1
    nbins = len(bins) - 1
    bin_idx = np.clip(bin_idx, 0, nbins-1)
    hist = np.bincount(bin_idx, weights=weights, minlength=nbins)
    w2 = np.bincount(bin_idx, weights=weights**2, minlength=nbins)
    return hist, w2

def chi2_between_histograms(data_hist, mc_hist, sigma_data=None, sigma_mc=None):
    """
    Compute chi2 = sum ((D - M)^2 / (sigma_D^2 + sigma_M^2))
    Provide sigma arrays or compute default (sqrt for data, sqrt(w2) for mc)
    """
    if sigma_data is None:
        sigma_data = np.sqrt(data_hist)
    if sigma_mc is None:
        # caller should provide w2->sqrt(w2) if needed
        raise ValueError("Please provide sigma_mc (sqrt of sum w^2 per bin).")
    denom = sigma_data**2 + sigma_mc**2 + 1e-12
    valid = denom > 0
    chi2 = np.sum(((data_hist[valid] - mc_hist[valid])**2) / denom[valid])
    return chi2

# ---------- Nominal training + closure (single run) ----------
# Build training arrays (control region selection: be careful to choose CR!)
# Here I assume exp_df and mc_df are already representing the control region.
X_exp = exp_df[vars_features].values
X_mc_train = mc_df[vars_features].values
w_mc_train = mc_df["weight"].values  # original mc weight

# optional: downsample if extremely large for speed (but keep sufficient statistics)
max_train = 200000
if len(X_exp) > max_train:
    idx = rng_global.choice(len(X_exp), max_train, replace=False)
    X_exp_train = X_exp[idx]
else:
    X_exp_train = X_exp

if len(X_mc_train) > max_train:
    idx2 = rng_global.choice(len(X_mc_train), max_train, replace=False)
    X_mc_train_small = X_mc_train[idx2]
    w_mc_train_small = w_mc_train[idx2]
else:
    X_mc_train_small = X_mc_train
    w_mc_train_small = w_mc_train

print("Training reweighter on:", X_exp_train.shape, "exp vs", X_mc_train_small.shape, "mc (weights sum {:.1f})".format(np.sum(w_mc_train_small)))

model, clf_base = train_reweighter(X_mc_train_small, X_exp_train, w_mc_train_small)

# apply to test set
X_mc_test = mc_test_df[vars_features].values[mask_test] if 'mask_test' in globals() else mc_test_df[vars_features].values
w_mc_test_orig = mc_test_df["weight"].values[mask_test] if 'mask_test' in globals() else mc_test_df["weight"].values
w_test_new, w_test_raw = get_reweight_from_model(model, X_mc_test, w_mc_test_orig, clip=(clip_min, clip_max))

# compute histograms and chi2
data_hist, _ = np.histogram(exp_df[closure_var].values, bins=rue_edges)
mc_hist_rew, w2 = weighted_hist_and_uncertainty(mc_test_df[closure_var].values[mask_test], w_test_new, rue_edges)
sigma_data = np.sqrt(data_hist)
sigma_mc_rew = np.sqrt(w2)
chi2_val = chi2_between_histograms(data_hist, mc_hist_rew, sigma_data=sigma_data, sigma_mc=sigma_mc_rew)
ndf = len(data_hist) - 1
pval = chi2.sf(chi2_val, ndf)
print("Nominal classifier reweight closure: chi2/ndf = {:.2f}/{}, p-value {:.3g}".format(chi2_val, ndf, pval))

# efficiency in interval
mask_data = (exp_df[closure_var].values > cut_low) & (exp_df[closure_var].values < cut_high)
eps_data = np.sum(mask_data) / np.sum(np.isfinite(exp_df[closure_var].values))
mask_test_cut = (mc_test_df[closure_var].values[mask_test] > cut_low) & (mc_test_df[closure_var].values[mask_test] < cut_high)
eps_mc_test_rew = np.sum(w_test_new[mask_test_cut]) / np.sum(w_test_new)
print("eff_data = {:.5f}, eff_mc_test_rew = {:.5f}, ratio MC/Data = {:.3f}".format(eps_data, eps_mc_test_rew, eps_mc_test_rew/eps_data))

# plot comparison
plt.figure(figsize=(8,4))
bin_centers = 0.5*(rue_edges[:-1] + rue_edges[1:])
plt.step(bin_centers, data_hist, where='mid', label='exp (counts)')
plt.step(bin_centers, mc_hist_rew, where='mid', label='mc_rew (weighted counts)')
plt.legend()
plt.ylabel("counts")
plt.xlabel(closure_var)
plt.title("Nominal classifier reweight closure")
plt.show()

# ---------- Bootstrap (index bootstrap for train sets) ----------
chi2s = []
effs = []
raw_weights_records = []  # optional if you want to inspect distribution of raw reweights (memory heavy)

for i in tqdm(range(Nboot), desc="bootstrap reweights"):
    # resample indices with replacement for training sets
    idx_exp_b = rng_global.integers(0, len(X_exp), size=len(X_exp))
    idx_mc_b = rng_global.integers(0, len(X_mc_train), size=len(X_mc_train))
    X_exp_b = X_exp[idx_exp_b]
    X_mc_b = X_mc_train[idx_mc_b]
    w_mc_b = w_mc_train[idx_mc_b]

    # train reweighter on bootstraped samples (we use smaller model to speed if needed)
    try:
        model_b, _ = train_reweighter(X_mc_b, X_exp_b, w_mc_b)
    except Exception as e:
        # fallback: skip this bootstrap
        continue

    # apply to test
    w_test_b, _ = get_reweight_from_model(model_b, X_mc_test, w_mc_test_orig, clip=(clip_min, clip_max))
    # hist and chi2
    mc_hist_b, w2_b = weighted_hist_and_uncertainty(mc_test_df[closure_var].values[mask_test], w_test_b, rue_edges)
    sigma_mc_b = np.sqrt(w2_b)
    chi2_b = chi2_between_histograms(data_hist, mc_hist_b, sigma_data=sigma_data, sigma_mc=sigma_mc_b)
    chi2s.append(chi2_b)
    # efficiency
    eff_b = np.sum(w_test_b[mask_test_cut]) / np.sum(w_test_b)
    effs.append(eff_b)

chi2s = np.array(chi2s)
effs = np.array(effs)
print("Bootstrap chi2 median {:.2f}, 16-84 pct: {:.2f} - {:.2f}".format(np.median(chi2s), np.percentile(chi2s,16), np.percentile(chi2s,84)))
print("Bootstrap eff median {:.5f}, 16-84 pct: {:.5f} - {:.5f}".format(np.median(effs), np.percentile(effs,16), np.percentile(effs,84)))

# Plot closure residual distribution for efficiency
deltas = eps_data - effs
plt.figure()
plt.hist(deltas, bins=40, density=True, histtype='step', color='red')
plt.axvline(0, color='k', linestyle='--')
plt.xlabel("closure residual (eps_data - eps_mc_rew)")
plt.ylabel("density")
plt.title("bootstrap closure residuals")
plt.show()

# ---------- Compare to histogram-based nominal (if you have nominal histogram wmap) ----------
# If you computed histogram-based MC weights earlier (nominal_wmap), show overlay for recE:
# compute mc_hist_histnom (apply nominal_wmap via bin indices precomputed similarly)
# (left as an exercise; use weighted_hist_and_uncertainty with weights from hist-based pipeline)


Training reweighter on: (200000, 3) exp vs (200000, 3) mc (weights sum 9131765.8)


TypeError: CalibratedClassifierCV.__init__() got an unexpected keyword argument 'base_estimator'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from tqdm import tqdm
from scipy.stats import chi2

# ---------- 配置区（按需修改） ----------
vars_features = ["r_Theta", "age", "dr"]   # 用于 reweight 的特征（control region）
label_exp = 1
label_mc = 0

# Classifier 超参（可调整）
clf_args = dict(n_estimators=300, learning_rate=0.1, max_depth=4, random_state=42)

# Calibration method: 'isotonic' or 'sigmoid'
calibration_method = 'isotonic'

# 权重剪切与归一化参数
clip_min = 1e-3
clip_max = 50.0   # 上限，防止单事件主导
normalize_mc_total = True   # 将 reweighted MC 保持与原始 MC total weight 相同（常用）

# bootstrap 设置
Nboot = 300
random_seed = 12345
n_jobs = 4   # 若使用并行训练的模型可设置

# closure 测试相关
closure_var = "recE"
rue_edges = np.linspace(np.percentile(exp_df[closure_var], 1),
                        np.percentile(exp_df[closure_var], 99), 50)
cut_low, cut_high = 1.79, 2.13
# ----------------------------------------

rng_global = np.random.default_rng(random_seed)

# ---------- 辅助函数 ----------
def train_reweighter(X_mc_train, X_exp_train, w_mc_train=None):
    """
    Train classifier to separate data vs mc.
    Inputs:
      X_mc_train: (N_mc_train, n_feat) numpy array
      X_exp_train: (N_exp_train, n_feat)
      w_mc_train: original mc weights for training (or None -> ones)
    Returns:
      calibrator: calibrated classifier object with predict_proba
      clf_base: raw classifier (optional)
    """
    X = np.vstack([X_exp_train, X_mc_train])
    y = np.concatenate([np.ones(len(X_exp_train)), np.zeros(len(X_mc_train))])
    # sample weights: data weight = 1, mc weight = w_mc_train (if provided)
    if w_mc_train is None:
        sw = np.concatenate([np.ones(len(X_exp_train)), np.ones(len(X_mc_train))])
    else:
        sw = np.concatenate([np.ones(len(X_exp_train)), w_mc_train])

    # train-test split for calibration set
    X_tr, X_val, y_tr, y_val, sw_tr, sw_val = train_test_split(
        X, y, sw, test_size=0.25, random_state=random_seed, stratify=y)

    clf = GradientBoostingClassifier(**clf_args)
    clf.fit(X_tr, y_tr, sample_weight=sw_tr)

    # calibrate probabilities on validation set
    calibrator = CalibratedClassifierCV(base_estimator=clf, method=calibration_method, cv='prefit')
    calibrator.fit(X_val, y_val, sample_weight=sw_val)
    return calibrator, clf

def get_reweight_from_model(model, X_mc_eval, w_mc_orig=None, clip=(clip_min, clip_max), normalize=normalize_mc_total):
    """
    Compute event-wise reweight for MC eval set.
    reweight = p/(1-p) where p = model.predict_proba(X)[:,1]
    """
    p = model.predict_proba(X_mc_eval)[:,1]
    eps = 1e-6
    p = np.clip(p, eps, 1-eps)
    w_rew = p / (1.0 - p)

    # optional: multiply by original mc weight if provided (we keep them separate)
    if w_mc_orig is None:
        w = w_rew
    else:
        w = w_rew * w_mc_orig

    # clip extreme values
    w = np.clip(w, clip[0], clip[1])

    if normalize and (w_mc_orig is not None):
        # keep total MC weight same as original total (shape correction only)
        total_orig = np.sum(w_mc_orig)
        total_new = np.sum(w)
        if total_new > 0:
            w = w * (total_orig / total_new)
    return w, w_rew  # return both final applied weights and raw reweight

def weighted_hist_and_uncertainty(vals, weights, bins):
    """
    Fast weighted histogram + per-bin sum of squared weights (for variance)
    Returns:
      hist (sum weights per bin), w2_per_bin (sum weights^2)
    """
    bin_idx = np.digitize(vals, bins) - 1
    nbins = len(bins) - 1
    bin_idx = np.clip(bin_idx, 0, nbins-1)
    hist = np.bincount(bin_idx, weights=weights, minlength=nbins)
    w2 = np.bincount(bin_idx, weights=weights**2, minlength=nbins)
    return hist, w2

def chi2_between_histograms(data_hist, mc_hist, sigma_data=None, sigma_mc=None):
    """
    Compute chi2 = sum ((D - M)^2 / (sigma_D^2 + sigma_M^2))
    Provide sigma arrays or compute default (sqrt for data, sqrt(w2) for mc)
    """
    if sigma_data is None:
        sigma_data = np.sqrt(data_hist)
    if sigma_mc is None:
        # caller should provide w2->sqrt(w2) if needed
        raise ValueError("Please provide sigma_mc (sqrt of sum w^2 per bin).")
    denom = sigma_data**2 + sigma_mc**2 + 1e-12
    valid = denom > 0
    chi2 = np.sum(((data_hist[valid] - mc_hist[valid])**2) / denom[valid])
    return chi2

# ---------- Nominal training + closure (single run) ----------
# Build training arrays (control region selection: be careful to choose CR!)
# Here I assume exp_df and mc_df are already representing the control region.
X_exp = exp_df[vars_features].values
X_mc_train = mc_df[vars_features].values
w_mc_train = mc_df["weight"].values  # original mc weight

# optional: downsample if extremely large for speed (but keep sufficient statistics)
max_train = 200000
if len(X_exp) > max_train:
    idx = rng_global.choice(len(X_exp), max_train, replace=False)
    X_exp_train = X_exp[idx]
else:
    X_exp_train = X_exp

if len(X_mc_train) > max_train:
    idx2 = rng_global.choice(len(X_mc_train), max_train, replace=False)
    X_mc_train_small = X_mc_train[idx2]
    w_mc_train_small = w_mc_train[idx2]
else:
    X_mc_train_small = X_mc_train
    w_mc_train_small = w_mc_train

print("Training reweighter on:", X_exp_train.shape, "exp vs", X_mc_train_small.shape, "mc (weights sum {:.1f})".format(np.sum(w_mc_train_small)))

model, clf_base = train_reweighter(X_mc_train_small, X_exp_train, w_mc_train_small)

# apply to test set
X_mc_test = mc_test_df[vars_features].values[mask_test] if 'mask_test' in globals() else mc_test_df[vars_features].values
w_mc_test_orig = mc_test_df["weight"].values[mask_test] if 'mask_test' in globals() else mc_test_df["weight"].values
w_test_new, w_test_raw = get_reweight_from_model(model, X_mc_test, w_mc_test_orig, clip=(clip_min, clip_max))

# compute histograms and chi2
data_hist, _ = np.histogram(exp_df[closure_var].values, bins=rue_edges)
mc_hist_rew, w2 = weighted_hist_and_uncertainty(mc_test_df[closure_var].values[mask_test], w_test_new, rue_edges)
sigma_data = np.sqrt(data_hist)
sigma_mc_rew = np.sqrt(w2)
chi2_val = chi2_between_histograms(data_hist, mc_hist_rew, sigma_data=sigma_data, sigma_mc=sigma_mc_rew)
ndf = len(data_hist) - 1
pval = chi2.sf(chi2_val, ndf)
print("Nominal classifier reweight closure: chi2/ndf = {:.2f}/{}, p-value {:.3g}".format(chi2_val, ndf, pval))

# efficiency in interval
mask_data = (exp_df[closure_var].values > cut_low) & (exp_df[closure_var].values < cut_high)
eps_data = np.sum(mask_data) / np.sum(np.isfinite(exp_df[closure_var].values))
mask_test_cut = (mc_test_df[closure_var].values[mask_test] > cut_low) & (mc_test_df[closure_var].values[mask_test] < cut_high)
eps_mc_test_rew = np.sum(w_test_new[mask_test_cut]) / np.sum(w_test_new)
print("eff_data = {:.5f}, eff_mc_test_rew = {:.5f}, ratio MC/Data = {:.3f}".format(eps_data, eps_mc_test_rew, eps_mc_test_rew/eps_data))

# plot comparison
plt.figure(figsize=(8,4))
bin_centers = 0.5*(rue_edges[:-1] + rue_edges[1:])
plt.step(bin_centers, data_hist, where='mid', label='exp (counts)')
plt.step(bin_centers, mc_hist_rew, where='mid', label='mc_rew (weighted counts)')
plt.legend()
plt.ylabel("counts")
plt.xlabel(closure_var)
plt.title("Nominal classifier reweight closure")
plt.show()

# ---------- Bootstrap (index bootstrap for train sets) ----------
chi2s = []
effs = []
raw_weights_records = []  # optional if you want to inspect distribution of raw reweights (memory heavy)

for i in tqdm(range(Nboot), desc="bootstrap reweights"):
    # resample indices with replacement for training sets
    idx_exp_b = rng_global.integers(0, len(X_exp), size=len(X_exp))
    idx_mc_b = rng_global.integers(0, len(X_mc_train), size=len(X_mc_train))
    X_exp_b = X_exp[idx_exp_b]
    X_mc_b = X_mc_train[idx_mc_b]
    w_mc_b = w_mc_train[idx_mc_b]

    # train reweighter on bootstraped samples (we use smaller model to speed if needed)
    try:
        model_b, _ = train_reweighter(X_mc_b, X_exp_b, w_mc_b)
    except Exception as e:
        # fallback: skip this bootstrap
        continue

    # apply to test
    w_test_b, _ = get_reweight_from_model(model_b, X_mc_test, w_mc_test_orig, clip=(clip_min, clip_max))
    # hist and chi2
    mc_hist_b, w2_b = weighted_hist_and_uncertainty(mc_test_df[closure_var].values[mask_test], w_test_b, rue_edges)
    sigma_mc_b = np.sqrt(w2_b)
    chi2_b = chi2_between_histograms(data_hist, mc_hist_b, sigma_data=sigma_data, sigma_mc=sigma_mc_b)
    chi2s.append(chi2_b)
    # efficiency
    eff_b = np.sum(w_test_b[mask_test_cut]) / np.sum(w_test_b)
    effs.append(eff_b)

chi2s = np.array(chi2s)
effs = np.array(effs)
print("Bootstrap chi2 median {:.2f}, 16-84 pct: {:.2f} - {:.2f}".format(np.median(chi2s), np.percentile(chi2s,16), np.percentile(chi2s,84)))
print("Bootstrap eff median {:.5f}, 16-84 pct: {:.5f} - {:.5f}".format(np.median(effs), np.percentile(effs,16), np.percentile(effs,84)))

# Plot closure residual distribution for efficiency
deltas = eps_data - effs
plt.figure()
plt.hist(deltas, bins=40, density=True, histtype='step', color='red')
plt.axvline(0, color='k', linestyle='--')
plt.xlabel("closure residual (eps_data - eps_mc_rew)")
plt.ylabel("density")
plt.title("bootstrap closure residuals")
plt.show()

# ---------- Compare to histogram-based nominal (if you have nominal histogram wmap) ----------
# If you computed histogram-based MC weights earlier (nominal_wmap), show overlay for recE:
# compute mc_hist_histnom (apply nominal_wmap via bin indices precomputed similarly)
# (left as an exercise; use weighted_hist_and_uncertainty with weights from hist-based pipeline)


In [6]:
exp_df = exp_df_sample
mc_df = mc_df_sample
mc_test_df = mc_test_df_sample
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
from scipy.stats import chi2

# ----------------- 可调参数（请在运行前检查/修改） -----------------
# variables to use for 3D reweight
vars_3d = ["r_Theta", "age", "dr"]

# percentile ranges to ignore extreme tails (avoid outliers controlling bins)
pct_low, pct_high = 2.5, 100

# desired approx number of bins per axis (fallback automatic)
nbins_per_axis = None  # if None, will be auto-chosen based on MC training sample size

# pseudo-count (Laplace prior) to add to hist bins to avoid zeros (small positive)
alpha = 1e-3  

# gaussian smoothing sigma (in bin units) applied on log(weight_map)
# If sigma=0 -> no smoothing. Typical 0.5~1.5
smooth_sigma = 1.0

# cap factor to avoid huge weights: w_cap = median_w * cap_factor
cap_factor = 10.0

# bootstrap reps
Nboot = 400

# cut to test closure on (example: R_ue < 0.5; change as needed)
cut_var = "recE"
cut_value_1 = 1.79
cut_value_2 = 2.13
# -----------------------------------------------------------------

# Helper: compute bin edges based on percentiles and nbins
def make_bin_edges(series, nbins, lowpct=pct_low, highpct=pct_high):
    lo = np.percentile(series, lowpct)
    hi = np.percentile(series, highpct)
    if lo == hi:
        lo -= 1e-6
        hi += 1e-6
    return np.linspace(lo, hi, nbins + 1)

# 1) choose nbins per axis if not provided
N_train = len(mc_df)
if nbins_per_axis is None:
    # rule-of-thumb: prefer nbins so that total bins ~ N_train / 50 (i.e. ~50 events per bin)
    # so nbins^3 ~ N_train/50 -> nbins ~ (N_train/50)**(1/3)
    approx = int(np.clip(round((N_train / 50.0) ** (1/3)), 20, 50))
    nbins_per_axis = max(4, approx)
print("Training events:", N_train, " -> nbins per axis:", nbins_per_axis)

# 2) build bin edges from exp_df (we want to map MC onto experimental support)
bin_edges = {}
for v in vars_3d:
    series = exp_df[v].dropna().values
    bin_edges[v] = make_bin_edges(series, nbins_per_axis)
    print(f"var {v}: bin range {bin_edges[v][0]:.3g} .. {bin_edges[v][-1]:.3g}")

# 3) function to compute 3D hist (weighted)
def hist3d(vals_dict, weights, edges):
    # vals_dict: dict var->1D array length N
    # edges: dict var->bin_edges
    h, _ = np.histogramdd(np.vstack([vals_dict[v] for v in vars_3d]).T,
                         bins=[edges[v] for v in vars_3d],
                         weights=weights, density=False)
    return h

# 4) train histograms (use exp_df as numerator, mc_df as denominator with mc weight included)
# Use only rows where all vars finite and within percentile range mapping
def select_and_extract(df):
    mask = np.ones(len(df), dtype=bool)
    arrs = {}
    for v in vars_3d:
        arr = df[v].values
        # clip to range (so binning consistent)
        lo, hi = bin_edges[v][0], bin_edges[v][-1]
        mask = mask & np.isfinite(arr) & (arr >= lo) & (arr <= hi)
        arrs[v] = arr
    return mask, arrs

mask_exp, arrs_exp_all = select_and_extract(exp_df)
mask_mc, arrs_mc_all = select_and_extract(mc_df)

print("Selected exp events:", mask_exp.sum(), " selected mc train events:", mask_mc.sum())

# numerator: experimental histogram (unweighted counts)
h_exp = hist3d({v: arrs_exp_all[v][mask_exp] for v in vars_3d},
               weights=np.ones(mask_exp.sum()), edges=bin_edges)

# denominator: mc train histogram using original mc weight
mc_weights_train = mc_df["weight"].values
h_mc = hist3d({v: arrs_mc_all[v][mask_mc] for v in vars_3d},
              weights=mc_weights_train[mask_mc], edges=bin_edges)

# 5) form raw weight_map = (h_exp + alpha) / (h_mc + alpha)
# add small alpha to numerator and denominator
h_exp_reg = h_exp + alpha
h_mc_reg = h_mc + alpha
raw_wmap = h_exp_reg / h_mc_reg

# 6) smoothing: work in log-space to avoid negative/zero issues
with np.errstate(divide='ignore'):
    log_w = np.log(raw_wmap)
# mask infinite entries (shouldn't happen with alpha) and apply gaussian filter
if smooth_sigma > 0:
    log_w_smooth = gaussian_filter(log_w, sigma=smooth_sigma, mode='nearest')
else:
    log_w_smooth = log_w.copy()
wmap = np.exp(log_w_smooth)

# 7) cap weights to avoid outliers
median_w = np.median(wmap[np.isfinite(wmap)])
w_cap = median_w * cap_factor
wmap_capped = np.where(wmap > w_cap, w_cap, wmap)

# Save nominal wmap and info
nominal_wmap = wmap_capped.copy()
print("wmap stats: median {:.3g}, mean {:.3g}, max {:.3g}, cap {:.3g}".format(
    np.median(nominal_wmap), np.mean(nominal_wmap), np.max(nominal_wmap), w_cap))

# ---------------------------------------------------------------------
# 8) apply nominal wmap to independent MC test set and compute closure metrics
# ---------------------------------------------------------------------
# prepare test arrays
mask_test, arrs_test_all = select_and_extract(mc_test_df)
print("Selected mc_test events:", mask_test.sum())

# get bin indices for mc_test
coords_test = np.vstack([arrs_test_all[v][mask_test] for v in vars_3d]).T
bin_idx_test = np.stack([
    np.digitize(arrs_test_all[v][mask_test], bin_edges[v]) - 1 for v in vars_3d
], axis=1)
# clip indices
for j, v in enumerate(vars_3d):
    bin_idx_test[:, j] = np.clip(bin_idx_test[:, j], 0, nominal_wmap.shape[j]-1)

# map each test event to bin weight
w_orig_test = mc_test_df["weight"].values[mask_test]
wmap_flat_index = bin_idx_test[:,0]* (nominal_wmap.shape[1]*nominal_wmap.shape[2]) + \
                  bin_idx_test[:,1]* nominal_wmap.shape[2] + bin_idx_test[:,2]
# faster: index via tuple
event_w_rew = nominal_wmap[bin_idx_test[:,0], bin_idx_test[:,1], bin_idx_test[:,2]]
# final test event weight = original mc weight * reweight factor
w_test_final = w_orig_test * event_w_rew

# build reweighted test histogram of R_ue to compare with experimental R_ue (we'll use same binning)
# choose R_ue bins (percentile-based on exp)
rue_edges = np.linspace(np.percentile(exp_df[cut_var], 1), np.percentile(exp_df[cut_var], 99), 50)
data_hist_rue, _ = np.histogram(exp_df[cut_var].values, bins=rue_edges, density=False)
mc_test_hist_rew, _ = np.histogram(mc_test_df[cut_var].values[mask_test], bins=rue_edges, weights=w_test_final, density=False)
mc_test_hist_orig, _ = np.histogram(mc_test_df[cut_var].values[mask_test], bins=rue_edges, weights=w_orig_test, density=False)

# compute per-bin uncertainties (data sqrt(N), weighted mc sqrt(sum w^2))
sigma_data = np.sqrt(data_hist_rue)
# compute sum of squared weights per bin
w2_sums = np.zeros_like(mc_test_hist_rew)
bin_idx_rue = np.digitize(mc_test_df[cut_var].values[mask_test], rue_edges) - 1
for b in range(len(mc_test_hist_rew)):
    sel = bin_idx_rue == b
    w2_sums[b] = np.sum(w_test_final[sel]**2)
sigma_mc_rew = np.sqrt(w2_sums)
sigma_tot = np.sqrt(sigma_data**2 + sigma_mc_rew**2 + 1e-12)
pulls = (data_hist_rue - mc_test_hist_rew) / sigma_tot
chi2_val = np.sum(((data_hist_rue - mc_test_hist_rew)**2) / (sigma_tot**2))
ndf = len(data_hist_rue) - 1
pval = chi2.sf(chi2_val, ndf)
print("Nominal closure: chi2/ndf = {:.2f}/{}, p-value {:.3g}".format(chi2_val, ndf, pval))
print("pull mean {:.3f}, std {:.3f}".format(np.nanmean(pulls), np.nanstd(pulls)))

# # efficiency for cut example: compute passing fraction for mc_test_rew vs exp (in control region)
# exp_mask_for_eff = np.isfinite(exp_df[cut_var].values)
# mask1=(exp_df[cut_var].values[exp_mask_for_eff] < cut_value_2)&(exp_df[cut_var].values[exp_mask_for_eff] > cut_value_1)
# eps_data = np.sum(mask1) / np.sum(exp_mask_for_eff)
# mask2=w_test_final[(mc_test_df[cut_var].values[mask_test] < cut_value_2)&(mc_test_df[cut_var].values[mask_test] > cut_value_1)]
# eps_mc_test = np.sum(mask2) / np.sum(w_test_final)
# print("eff_data = {:.4%}, eff_mc_test_rew = {:.4%}".format(eps_data, eps_mc_test))

# # ---------------------------------------------------------------------
# # 9) Bootstrap: resample training sets and recompute wmap each time, apply to test, collect metrics
# # ---------------------------------------------------------------------
# def compute_wmap_from_train(train_mc_df, train_exp_df):
#     # build hist from provided training samples (must be preselected within edges)
#     mask_t_mc, arrs_t_mc = select_and_extract(train_mc_df)
#     mask_t_exp, arrs_t_exp = select_and_extract(train_exp_df)
#     h_exp_t = hist3d({v: arrs_t_exp[v][mask_t_exp] for v in vars_3d},
#                      weights=np.ones(mask_t_exp.sum()), edges=bin_edges)
#     h_mc_t = hist3d({v: arrs_t_mc[v][mask_t_mc] for v in vars_3d},
#                     weights=train_mc_df["weight"].values[mask_t_mc], edges=bin_edges)
#     h_exp_t += alpha
#     h_mc_t += alpha
#     raw = h_exp_t / h_mc_t
#     lograw = np.log(raw)
#     smooth = gaussian_filter(lograw, sigma=smooth_sigma, mode='nearest')
#     w = np.exp(smooth)
#     # cap
#     med = np.median(w[np.isfinite(w)])
#     cap = med * cap_factor
#     w = np.where(w > cap, cap, w)
#     return w

# # prepare arrays to store boot results
# chi2s = []
# effs_mc = []
# # optionally store per-bin distributions (heavy mem) - we will store percentiles instead
# # for performance, precompute test indices arrays
# test_vals_arr = mc_test_df[cut_var].values[mask_test]
# test_mask_len = mask_test.sum()
# # Precompute: test bin index (for closure variable)
# data_vals = exp_df[cut_var].values
# test_vals = mc_test_df[cut_var].values[mask_test]
# bin_idx_rue = np.digitize(test_vals, rue_edges) - 1
# Nbins = len(rue_edges) - 1

# def fast_sigma_mc(w):
#     # bincount is much faster than loop
#     return np.sqrt(np.bincount(bin_idx_rue, weights=w*w, minlength=Nbins))


# chi2s = []
# effs_mc = []
# rng = np.random.default_rng(12345)
# for i in tqdm(range(Nboot)):
#     # Resample
#     mc_boot = mc_df.sample(frac=1, replace=True, random_state=rng.integers(1e9))
#     exp_boot = exp_df.sample(frac=1, replace=True, random_state=rng.integers(1e9))
#     # Compute wmap
#     wmap_b = compute_wmap_from_train(mc_boot, exp_boot)
#     # Apply
#     w_test_b = w_orig_test * wmap_b[bin_idx_test[:,0], bin_idx_test[:,1], bin_idx_test[:,2]]
#     # Compute histogram (counts)
#     mc_hist_b, _ = np.histogram(test_vals, bins=rue_edges, weights=w_test_b)
#     # Uncertainty
#     sigma_mc_b = fast_sigma_mc(w_test_b)
#     sigma_tot_b = np.sqrt(sigma_data**2 + sigma_mc_b**2 + 1e-12)
#     # χ²
#     chi2_b = np.sum((data_hist_rue - mc_hist_b)**2 / (sigma_tot_b**2))
#     chi2s.append(chi2_b)
#     # Efficiency
#     eff_mask = (test_vals < cut_value_2) & (test_vals > cut_value_1)
#     eff_b = np.sum(w_test_b[eff_mask]) / np.sum(w_test_b)
#     effs_mc.append(eff_b)

# chi2s = np.array(chi2s)
# effs_mc = np.array(effs_mc)

# print("Bootstrap chi2 median {:.2f}, 16-84: {:.2f}-{:.2f}".format(np.median(chi2s), np.percentile(chi2s,16), np.percentile(chi2s,84)))
# print("Bootstrap eff_mc median {:.4f}, 16-84: {:.4f}-{:.4f}".format(np.median(effs_mc), np.percentile(effs_mc,16), np.percentile(effs_mc,84)))

# # Compute closure residual distribution: delta = eps_data - eps_mc_boot
# deltas = eps_data - effs_mc
# print("closure delta median {:.4f}, 16-84: {:.4f}-{:.4f}".format(np.median(deltas), np.percentile(deltas,16), np.percentile(deltas,84)))


# plt.hist(deltas, bins=20, density=True, histtype="step", color="red")
# plt.axvline(0, color="k", linestyle="--")
# plt.xlabel("closure residual")
# plt.ylabel("density")
# plt.savefig("./figures/closure_residual.png")
# plt.show()
# interpret: if distribution centered near 0 and spread small -> good closure
# otherwise take spread (e.g., 68% half-range) and/or median bias as modeling uncertainty

# Save nominal wmap and percentiles (e.g., per bin 16/50/84) for shape nuisance creation if needed:
# to compute percentiles per bin we'd need to store each boot wmap; that is memory heavy but doable if nbins small.


Training events: 1000000  -> nbins per axis: 27
var r_Theta: bin range 0.0715 .. 1.55
var age: bin range 0.632 .. 2.5
var dr: bin range -32 .. 235
Selected exp events: 940004  selected mc train events: 921188
wmap stats: median 0.0423, mean 0.15, max 0.423, cap 0.423
Selected mc_test events: 921206
Nominal closure: chi2/ndf = 40992.70/48, p-value 0
pull mean 18.740, std 22.032


In [8]:
dr_ratio_exp = np.sum(exp_df["dr"] > 50) / exp_df.shape[0]
dr_ratio_mc = np.sum(mc_df["dr"] > 50) / mc_df.shape[0]
print(f"Exp dr > 50: {dr_ratio_exp}")
print(f"MC dr > 50: {dr_ratio_mc}")
print(f"diff: {dr_ratio_mc - dr_ratio_exp}")

age_radio_exp = np.sum((exp_df["age"]>0.8)&(exp_df["age"]<1.2)) / exp_df.shape[0]
age_radio_mc = np.sum((mc_df["age"]>0.8)&(mc_df["age"]<1.2)) / mc_df.shape[0]
print(f"Exp age: {age_radio_exp}")
print(f"MC age: {age_radio_mc}")
print(f"diff: {age_radio_mc - age_radio_exp}")

r_Theta_radio_exp = np.sum((exp_df["r_Theta"]<np.pi*50/180)) / exp_df.shape[0]
r_Theta_radio_mc = np.sum((mc_df["r_Theta"]<np.pi*50/180)) / mc_df.shape[0]
print(f"Exp r_Theta: {r_Theta_radio_exp}")  
print(f"MC r_Theta: {r_Theta_radio_mc}")
print(f"diff: {r_Theta_radio_mc - r_Theta_radio_exp}")


Exp dr > 50: 0.7543098076909313
MC dr > 50: 0.6934233121082676
diff: -0.06088649558266368
Exp age: 0.11434744077358555
MC age: 0.1932981042105104
diff: 0.07895066343692486
Exp r_Theta: 0.9804395836029609
MC r_Theta: 0.9313842408349879
diff: -0.049055342767972965
